# Configuration 

In [ ]:
import os
from openai import OpenAI



from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()



AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = "gpt-4o-mini"  

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT


# Create client for Azure OpenAI
client = OpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    base_url=f"{os.environ['AZURE_OPENAI_ENDPOINT']}/openai/v1/",
)

# Simple test call
resp = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT, 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Introduce your self in short sentence."},
    ],
)

print(resp.choices[0].message.content)


# Check the language syntax 

In [ ]:
def normalize_query(text: str) -> str:
    resp = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You understand poorly written Norwegian and English. Correct spelling mistakes, interpret the meaning, and rewrite the sentence clearly."},
            {"role": "user", "content": text},
        ],
        temperature=0.1
    )
    return resp.choices[0].message.content.strip()


# LLM planner: convert the natural language query to a structured JSON plan


In [ ]:
import json

SYSTEM_PROMPT = """


You are a GIS planner agent for a Nordic municipality.
You NEVER execute SQL and NEVER touch the database.
You ONLY output a JSON plan that another system will translate to SQL.

------------------------------------------------------------
OUTPUT FORMAT (STRICT)
------------------------------------------------------------
You MUST output ONLY valid JSON with EXACTLY these fields:

{
  "operation": "...",
  "layer": "...",
  "target_layer": "...",
  "buffer_meters": ...,
  "limit": ...,
  "where_clause": "..."
}




Rules:
- No backticks, no explanations, no comments.
- All fields MUST exist, even if null or empty.
- buffer_meters MUST be a number or null.
- limit MUST be a number or null.
- where_clause MUST be either "" or a simple phrase (see rules below).
- layer and target_layer MUST be valid or "".


------------------------------------------------------------
ALLOWED OPERATIONS
------------------------------------------------------------
General:
- "select_limit_only"
- "select_by_attribute"
- "select_buffer"
- "select_intersect"
- "select_nearest"
- "select_within_polygon"

Special (only if user explicitly asks):
- Buildings: "select_buildings_in_floodzone", "select_buildings_near_route", "select_buildings_by_area"
- Flood zones: "select_within_floodzone", "select_intersect_floodzone"
- Bicycle routes: "select_near_bikeroute", "select_intersect_bikeroute"
- Walking routes: "select_near_walkroute", "select_intersect_walkroute"
- Ski routes: "select_near_skiroute", "select_intersect_skiroute"
- Route info points: "select_nearest_rutepoint", "select_points_in_area"



------------------------------------------------------------
ALLOWED LAYERS (MUST MATCH EXACTLY)
------------------------------------------------------------
- "buildings"
- "flomsoner"
- "buildings_sample"
- "arealbruk_skogbonitet_sample"
- "flomsoner_sample"
- "sykkelrute_senterlinje_sample"
- "skiloype_senterlinje"
- "annenrute_senterlinje"
- "annenruteinfo_tabell"
- "arealbruk_skogbonitet"
- "fotrute_senterlinje"
- "fotruteinfo_tabell"
- "ruteinfopunkt_posisjon"
- "skiloypeinfo_tabell"
- "sykkelrute_senterlinje"
- "sykkelruteinfo_tabell"

If no layer is clearly referenced → layer = "".






------------------------------------------------------------
target_layer RULES (VERY IMPORTANT)
------------------------------------------------------------
For ANY spatial operation (buffer, intersect, nearest),
the JSON MUST include a valid "target_layer".

Mapping:
- "near water" → "flomsoner"
- "near river" → "flomsoner"
- "intersect floodzone" → "flomsoner"
- "inside floodzone" → "flomsoner"
- "near bikeroute" → "sykkelrute_senterlinje"
- "intersect bikeroute" → "sykkelrute_senterlinje"
- "near walkroute" → "fotrute_senterlinje"
- "near skiroute" → "skiloype_senterlinje"

If user gives NO spatial relation → target_layer MUST be "".



------------------------------------------------------------
OPERATION SELECTION RULES
------------------------------------------------------------
- If the user says “within X meters” → operation = "select_buffer".
- If the user says “near X” → operation = "select_buffer", unless they say “nearest”.
- If the user says “nearest” or “closest” → operation = "select_nearest".
- If no spatial relation is described → DO NOT choose buffer/intersect/nearest.
- City names (e.g., “Kristiansand”) MUST NOT produce filters.

------------------------------------------------------------
LIMIT RULES
------------------------------------------------------------
- If user gives a number (e.g., “10 buildings”) → use it.
- If user does not specify → limit = null.

------------------------------------------------------------
BUFFER RULES
------------------------------------------------------------
- If user says “within X meters”, set buffer_meters = X.
- Otherwise buffer_meters = null.

------------------------------------------------------------
WHERE_CLAUSE RULES
------------------------------------------------------------
Only fill where_clause if the user explicitly says:
- near water
- near river
- near bikeroute
- inside floodzone
- intersect floodzone
- etc.

Otherwise: where_clause = "".

------------------------------------------------------------
FINAL INSTRUCTION
------------------------------------------------------------
Your entire output MUST be a single JSON object following all rules.
No prose. No Markdown. No explanations. Only JSON.

"""


previous_input = None

def plan_spatial_query(nl_query: str) -> dict:
    resp = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": nl_query},
        ],
        temperature=0.0,
        max_tokens=300,
    )
    raw = resp.choices[0].message.content.strip()
    if raw.startswith("```"):
        raw = raw.strip("`")
        if raw.lower().startswith("json"):
            raw = raw[4:].strip()
    return json.loads(raw)



# Process user input

In [ ]:
FIELD_INFO = {
    # ****************************************************
    "operation": (
        "Type of GIS action.\n"
        "Examples:\n"
        "- select_buffer\n"
        "- select_intersect\n"
        "- select_nearest\n"
        "- select_limit_only\n"
        "- select_by_attribute\n"
        "This tells the GIS engine WHAT to do."
    ),

    # ****************************************************
    "layer": (
        "The PRIMARY dataset you want results from.\n"
        "Must match one of the allowed database layers:\n"
        "- buildings\n"
        "- flomsoner\n"
        "- buildings_sample\n"
        "- arealbruk_skogbonitet_sample\n"
        "- flomsoner_sample\n"
        "- sykkelrute_senterlinje_sample\n"
        "- skiloype_senterlinje\n"
        "- annenrute_senterlinje\n"
        "- annenruteinfo_tabell\n"
        "- arealbruk_skogbonitet\n"
        "- fotrute_senterlinje\n"
        "- fotruteinfo_tabell\n"
        "- ruteinfopunkt_posisjon\n"
        "- skiloypeinfo_tabell\n"
        "- sykkelrute_senterlinje\n"
        "- sykkelruteinfo_tabell"
    ),

    # ****************************************************
    "target_layer": (
        "The SECOND dataset used for spatial relation.\n"
        "ONLY required for operations involving two layers (buffer, intersect, nearest).\n"
        "Examples:\n"
        "- layer = buildings\n"
        "- target_layer = sykkelrute_senterlinje  -> buildings near bike routes\n"
        "- target_layer = flomsoner               -> buildings intersect floodzones\n"
        "\nIf operation does not require a second dataset → target_layer must be null."
    ),

    # ****************************************************
    "buffer_meters": (
        "Distance in meters for spatial proximity.\n"
        "Examples:\n"
        "- 50\n"
        "- 100\n"
        "- 200\n"
        "Used only for operations requiring distance (select_buffer, select_near_...)."
    ),

    # ****************************************************
    "limit": (
        "How many results to return.\n"
        "Examples: 5, 10, 50.\n"
        "If not provided by the user → limit = null.\n"
        "If the user writes 'all' → limit will automatically become 100."
    ),


    # ****************************************************
    "where_clause": (
        "Optional simple spatial or attribute filter.\n"
        "Only filled when the user explicitly mentions a real filter:\n"
        "- near river\n"
        "- inside floodzone\n"
        "- intersect bikeroute\n"
        "- type = 'house'\n"
        "\nIf user does NOT specify a filter → where_clause = \"\"."
    )
}



def process_user_input(user_input):
    
    # Normalize language
    clean_text = normalize_query(user_input)

    # Create JSON plan
    plan = plan_spatial_query(clean_text)

    
    
    # for f in plan: 
    #     print(f,"is " ,plan.get(f),  "\n")

    

    #  Validate plan
    required_fields = ["operation", "layer", "buffer_meters", "limit", "target_layer"]
    missing = []


    limit = plan.get["limit"]
    
    if limit.lower() == "all": 
        plan["limit"] = 100

    
    for f in required_fields: 
        if ( plan[f] == None or plan[f] == ""): 
            missing.append(f)

    

    if missing:
        explanations = "\n".join([f"- {f}: {FIELD_INFO[f]}" for f in missing])
        return (
            
            f"\n⚠ Missing required field(s): {', '.join(missing)}.\n \n"

            "'''''''''''''''''''''''''''''''\n"

            f"{explanations}\n"
            f"\n"
            "------------------------------\n"
            f"\n{user_input}\n"
            f"\n"
            "'''''''''''''''''''''''''''''''\n"
            
        )

    # 5) If all good → return plan
    return plan


# Test the LLM planner

In [ ]:

query = "Find 100 residential houses within 200 meters of a river in Kristiansand."
plan = plan_spatial_query(query)
plan


In [ ]:

query = "Find all buildings within 100 m of bicycle routes"
plan = plan_spatial_query(query)
plan


# Converts the user’s prompt into an SQL query string.

In [ ]:
def plan_to_sql(plan: dict) -> str:
    op = plan["operation"]
    layer = plan["layer"]
    target = plan.get("target_layer")  
    buf = plan.get("buffer_meters")
    limit = plan.get("limit") or 100
    where = plan.get("where_clause") or "TRUE"

    # SELECT LIMIT ONLY
    if op == "select_limit_only":
        return f"""
SELECT *
FROM public.{layer}
LIMIT {limit};
""".strip()

    # SELECT BY ATTRIBUTE
    if op == "select_by_attribute":
        return f"""
SELECT *
FROM public.{layer}
WHERE {where}
LIMIT {limit};
""".strip()

    # BUFFER OPERATION (A near B)
    if op == "select_buffer":
        if not target:
            raise ValueError("select_buffer requires 'target_layer'")
        return f"""
SELECT a.*
FROM public.{layer} a
JOIN (
    SELECT ST_Buffer(geom, {buf}) AS geom
    FROM public.{target}
) t
ON ST_Intersects(a.geom, t.geom)
WHERE {where}
LIMIT {limit};
""".strip()

    # INTERSECT OPERATION (A intersects B)
    if op == "select_intersect":
        if not target:
            raise ValueError("select_intersect requires 'target_layer'")
        return f"""
SELECT a.*
FROM public.{layer} a
JOIN public.{target} b
  ON ST_Intersects(a.geom, b.geom)
WHERE {where}
LIMIT {limit};
""".strip()

    # NEAREST OPERATION (A nearest to B)
    if op == "select_nearest":
        if not target:
            raise ValueError("select_nearest requires 'target_layer'")
        return f"""
SELECT a.*
FROM public.{layer} a
ORDER BY (
    SELECT MIN(ST_Distance(a.geom, b.geom))
    FROM public.{target} b
)
LIMIT {limit};
""".strip()

    # WITHIN POLYGON
    if op == "select_within_polygon":
        polygon = plan["where_clause"]
        return f"""
SELECT *
FROM public.{layer}
WHERE ST_Within(geom, ST_GeomFromText('{polygon}', 4326))
LIMIT {limit};
""".strip()

    raise ValueError(f"Unsupported operation: {op}")


# Connect to the PostGIS database and run the SQL query, returning a DataFrame


In [ ]:
import psycopg2
import pandas as pd

def run_postgis_query(sql: str) -> pd.DataFrame:
    conn_str = os.environ["PGCONN_STRING"]
    
    
    with psycopg2.connect(conn_str) as conn:
        
        with conn.cursor() as cur:
            cur.execute(sql)
            rows = cur.fetchall()
            cols = [desc[0] for desc in cur.description]
    return pd.DataFrame(rows, columns=cols)


# Execute the GIS plan in PostGIS and return the result as a DataFrame


In [ ]:
def execute_gis_plan_db(plan: dict):
    sql = plan_to_sql(plan)
    df = run_postgis_query(sql)
    return df

# Ask the GIS agent: NL query -> LLM plan -> SQL -> PostGIS -> DataFrame


In [ ]:
def ask_gis_agent(query: str) -> pd.DataFrame:



    
    
    plan = process_user_input(query)

    # If the agent returned a string → it's an error message
    if isinstance(plan, str):
        print(plan)
        return None  # STOP HERE

    
    df = execute_gis_plan_db(plan)
    return df

# Send the query to the GIS agent and display the first results


In [ ]:

queryy = "Find 100 residential houses within 200 meters of a river in Kristiansand."
plan = plan_spatial_query(query)
plan


In [ ]:

queryy = "Find all buildings within 100 m of bicycle routes."
plan = plan_spatial_query(query)
plan


In [ ]:
query = "Find 100 residential houses within 200 meters of a river in Kristiansand."
df = ask_gis_agent(query)
df


In [ ]:
 res = ask_gis_agent("Find all buildings within 100 m of bicycle routes")
res

In [ ]:
df.head()

# Simple CLI chat loop that sends user queries to the GIS agent and prints the results


In [ ]:
def chat_loop():
    # previous_input = None
    print("GIS agent chat – type 'quit' to stop.\n")

    while True:
        user_q = input("You: ").strip()
        
        print("'''''''''''''''''''''''''''''''''''''''''''")
    


        if user_q.lower() in ("quit", "exit", "q"):
            print("Welcome back")
            break


        # if user_q in ["new", "restart", "rest"]:
        #     previous_input = None
        #     continue

        # # If we had missing fields earlier → append the new text
        # if previous_input is not None:
        #     combined = previous_input + " " + user_q
        #     user_q = combined

        result = ask_gis_agent(user_q)

        #  Missing fields → return string warning
        if isinstance(result, str):
            print(result + "\n")
            previous_input = user_q  # STORE it for next turn
            continue
         
        print(result)

        # previous_input = None  # reset when success
chat_loop()

In [ ]:
from IPython.display import display




def chat_loop():
    print("GIS agent chat – type 'quit' to stop.\n")
    while True:
        user_q = input("You: ")
        if user_q.strip().lower() in ("quit", "exit", "q"):
            print(" ")
            break

        try:
            df = ask_gis_agent(user_q)
            if len(df) == 0:
                print("No results found.\n")
            else:
                display(df)  
        except Exception as e:
            print("Error:", e, "\n")

chat_loop()


In [ ]:
import psycopg2
import pandas as pd

def run_postgis_query(sql: str) -> pd.DataFrame:
    conn_str = os.environ["PGCONN_STRING"]
    
    
    with psycopg2.connect(conn_str) as conn:
        
        with conn.cursor() as cur:
            cur.execute(sql)
            rows = cur.fetchall()
            cols = [desc[0] for desc in cur.description]
    return pd.DataFrame(rows, columns=cols)


In [ ]:
run_postgis_query("""
SELECT table_schema, table_name 
FROM information_schema.tables 
WHERE table_schema='public';
""")


In [ ]:
run_postgis_query("""
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name='buildings';
""")


In [ ]:
run_postgis_query("SELECT * FROM buildings LIMIT 50;")


In [ ]:
print

In [ ]:
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'flomsoner';


In [ ]:
run_postgis_query("""
SELECT b.*
FROM public.buildings b
JOIN public.sykkelrute_senterlinje r
  ON ST_DWithin(b.geom, r.geom, 100)
LIMIT 50;
""")


In [ ]:
run_postgis_query("SELECT * FROM sykkelrute_senterlinje LIMIT 50;")


In [ ]:

run_postgis_query("SELECT * FROM flomsoner LIMIT 50;")
